In [14]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
import pickle
import mlflow

In [2]:
mlflow.set_tracking_uri('sqlite:///mlflow.db') # set the database
mlflow.set_experiment('my-brand-new-experiment') # create the experiment if not exists

In [3]:
def read_dataframe(path):
  df = pd.read_parquet(path)
  df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
  df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
  df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.seconds / 60

  df = df[((df.duration >=1) & (df.duration <=60))]
  return df

In [4]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

valid_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(valid_dicts)

In [9]:
target = 'duration'

y_train = df_train[target].values
y_val = df_val[target].values

In [23]:
with mlflow.start_run():

            mlflow.set_tag("developer", "patrick") 
            mlflow.log_param("train-data-path", "green_trip_data_2021-01.csv")
            mlflow.log_param("train-val-path", "green_trip_data_2021-02.csv")

            alpha = 0.005
            mlflow.log_param("alpha", alpha)

            lr = Lasso(alpha)
            lr.fit(X_train, y_train)

            y_pred = lr.predict(X_val)

            rmse = mean_squared_error(y_val, y_pred, squared=False)
            mlflow.log_param("rmse", rmse)
            
            dv_path = 'models/preprocessor.bin'
            with open(dv_path, 'wb') as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact(dv_path, artifact_path='preprocessing')
            mlflow.sklearn.log_model(lr, artifact_path='models_pickle')

In [32]:
logged_model = "runs:/aee138419727487aaa3f839edca95118/models_pickle"
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [33]:
loaded_model.predict(X_val)

array([16.73135866, 14.88594911, 21.60303965, ..., 10.4921909 ,
       10.06259391,  8.9003303 ])